## IMPORTant Stuff

In [ ]:
import geopandas as gp
import folium

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import glob
import functools

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

In [4]:
pd.set_option('display.max_columns', None)


In [2]:
kick_data = pd.read_csv('../../data/02_intermediate/kick_inter.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11,15,16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
kick_data.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'last_update_published_at', 'launched_at', 'name', 'permissions',
       'pledged', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'unread_messages_count',
       'unseen_activity_count', 'urls', 'usd_pledged', 'usd_type',
       'sub_category', 'overall_category', 'city', 'country_loc', 'state_loc',
       'creator_name', 'creator_slug'],
      dtype='object')

In [25]:
kick_data.state.value_counts()

successful    139265
failed        121385
live           52687
canceled       15657
suspended       1018
Name: state, dtype: int64

Dropping data on "live, canceled, and suspended" campaigns. (Maybe plot LIVE separately..or same plot with different color marker?

In [30]:
kick_to_plot = kick_data.loc[((kick_data.state == 'successful') | (kick_data.state == 'failed') | (kick_data.state == 'live'))]

In [9]:
kick_data.loc[(kick_data.country_loc=='US')].city.nunique() # Number of unique US cities in dataset

6919

In [14]:
kick_data.city.value_counts()

Los Angeles              9072
New York                 7657
London                   6914
Chicago                  3867
San Francisco            3292
Brooklyn                 3099
Portland                 2716
Seattle                  2635
Atlanta                  2155
Toronto                  2090
Boston                   2001
Austin                   1990
San Diego                1662
Philadelphia             1661
Minneapolis              1461
Denver                   1429
Washington               1422
Houston                  1384
Nashville                1380
Dallas                   1377
Vancouver                1329
Hong Kong                1234
Melbourne                1221
Paris                    1164
Salt Lake City           1110
Orlando                  1101
Manhattan                1078
Las Vegas                1062
Sydney                   1042
Phoenix                  1022
                         ... 
Denekamp                    1
Chimaltenango               1
Lauderdale

There are 6919 unique US cities represented in our dataframe. Using https://simplemaps.com/data/us-cities, I will aim to get the Long/Lat info for each city and add to this df. 

In [10]:
cities = pd.read_csv('../../data/01_raw/uscitiesv1.5.csv')

In [15]:
cities.loc[(cities.city == 'Los Angeles')]

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
8446,Los Angeles,Los Angeles,TX,Texas,48283,La Salle,28.4658,-99.0000,NaN,NaN,3.5,point,False,America/Chicago,78021,1840036276
16938,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12815475.0,3999759.0,3295.0,polygon,True,America/Los_Angeles,90291 90293 90292 91316 91311 90034 90037 9000...,1840020491


Thinking about the mask I would need to place on my dataframe. 

I need just US + unique City/State combinations. The latter 

In [16]:
kick_data.country_loc.isnull().sum() ## 1102 rows do not have a country defined - going to drop those.

1102

In [21]:
#kick_data.dropna(subset = ['country_loc'], inplace = True)

In [24]:
kick_data.state_loc.isnull().sum()    # 93
kick_data.city.isnull().sum()         # 133447
kick_to_plot.city.isnull().sum()      # 123743
kick_to_plot.state_loc.isnull().sum() # 86

93

In [39]:
kick_plot_US = kick_to_plot.loc[(kick_to_plot.country_loc == 'US')]

In [94]:
kick_plot_US.rename(columns = {'state_name':'state_id'}, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [41]:
drop = ['currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'is_backing', 'is_starrable', 'is_starred',
       'last_update_published_at', 'permissions', 'slug', 'static_usd_rate', 'unread_messages_count',
       'unseen_activity_count', 'urls', 'usd_type', 'creator_slug']

In [67]:
#kick_plot_US.drop(columns = drop, inplace = True)
kick_plot_US.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'goal', 'id', 'launched_at', 'name', 'pledged',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'usd_pledged', 'sub_category', 'overall_category', 'city',
       'country_loc', 'state_name', 'creator_name'],
      dtype='object')

In [56]:
kick_plot_US['launched_at'] = pd.to_datetime(kick_plot_US['launched_at'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
kick_plot_US['launched_at'] - kick_plot_US['created_at']

0           0 days 00:48:54
2           0 days 04:28:05
3           0 days 00:45:51
4           0 days 00:49:01
7           0 days 07:37:44
8           0 days 00:51:33
12          0 days 18:04:18
13          0 days 00:54:59
14          0 days 01:29:14
15          0 days 02:21:26
16          0 days 03:10:54
17          0 days 02:06:30
19          0 days 01:23:23
21          1 days 02:15:02
22          1 days 01:39:46
25          0 days 00:40:06
26          0 days 00:31:10
27          1 days 02:45:33
28          0 days 10:50:59
31          0 days 09:03:40
32          0 days 02:13:35
34          1 days 13:54:57
35          0 days 03:21:19
38          0 days 07:57:15
39          2 days 09:03:46
44          1 days 17:03:11
51          2 days 13:11:15
52          2 days 11:23:15
53          0 days 16:51:47
55          3 days 02:05:12
                ...        
331067      0 days 03:57:46
331068      0 days 01:05:04
331070      0 days 00:16:16
331071      2 days 14:58:38
331073     13 days 0

In [46]:
kick_plot_US['campaign_length'] = kick_plot_US.state_changed_at - kick_plot_US.launched_at

TypeError: unsupported operand type(s) for -: 'str' and 'str'

Trying to Merge Cities to Kick_Plot_US

### Random help for Misha

In [79]:
rand = {'key1':['value1', 'val2', 'val3'], 'key2': ['val4', 'val5']}

In [81]:
dict_new = {}
for key, val in rand.items():
    for i in range(len(val)):
        dict_new[val[i]] = key
        
print(dict_new)

{'value1': 'key1', 'val2': 'key1', 'val3': 'key1', 'val4': 'key2', 'val5': 'key2'}


### Now back to mapping these values in

Important lesson for merging tables: 
Was having issue with NaN values appearing in the 'lat','lng' columns. Turns out that in one DF, the "state_name" column had actual names, in the DF I was merging with "state_name" had the two-letter state ID. Therefore nothing was a complete match for a proper merge. 

In [98]:
kickUS_map = kick_plot_US.merge(cities[['city', 'state_id', 'lat', 'lng']], 
                   how = 'left', on = ['state_id', 'city'])

In [76]:
cities.loc[(cities.city == 'Cincinnati')]

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
24836,Cincinnati,Cincinnati,IA,Iowa,19007,Appanoose,40.6309,-92.9222,340.0,340.0,75.0,polygon,True,America/Chicago,52549,1840007203
29125,Cincinnati,Cincinnati,OH,Ohio,39061,Hamilton,39.1412,-84.5059,1648254.0,301301.0,1493.0,polygon,True,America/New_York,45219 45213 45212 45211 45217 45216 45214 4523...,1840003814


In [100]:
kickUS_map.state.value_counts()

successful    104150
failed         87907
live           31081
Name: state, dtype: int64

In [101]:
kickUS_map.to_pickle('../../data/02_intermediate/kickUS_map.pkl')